In [6]:
import requests 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'requests'

In [9]:
docsurl = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docsoutput=requests.get(docsurl)
docuraw = docsoutput.json()
documents = []

for course in docuraw:
        coursename = course['course']

        for doc in course['documents']:
            doc['course'] = coursename
            documents.append(doc)

In [5]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

NameError: name 'pd' is not defined

In [4]:
df[df.course == 'data-engineering-zoomcamp'].head()
documents = [
    "Course starts on 15th Jan 2024",
    "Prerequisites listed on GitHub",
    "Submit homeworks after start date",
    "Registration not required for participation",
    "Setup Google Cloud and Python before course"
]

NameError: name 'df' is not defined

In [12]:
cv = TfidfVectorizer(stop_words='english')
X = cv.fit_transform(documents)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T 
df_docs.round(2)

,0,1,2,3,4
15th,0.46,0.00,0.0,0.00,0.00
2024,0.46,0.00,0.0,0.00,0.00
cloud,0.00,0.00,0.0,0.00,0.46
course,0.37,0.00,0.0,0.00,0.37
date,0.00,0.00,0.5,0.00,0.00
github,0.00,0.58,0.0,0.00,0.00
google,0.00,0.00,0.0,0.00,0.46
homeworks,0.00,0.00,0.5,0.00,0.00
jan,0.46,0.00,0.0,0.00,0.00
listed,0.00,0.58,0.0,0.00,0.00


In [13]:
query = "Do I need to know python to sign up?"
q = cv.transform([query])
q.toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0.]])

In [14]:
query_dict = dict(zip(names, q.toarray()[0]))
query_dict

{'15th': 0.0,
 '2024': 0.0,
 'cloud': 0.0,
 'course': 0.0,
 'date': 0.0,
 'github': 0.0,
 'google': 0.0,
 'homeworks': 0.0,
 'jan': 0.0,
 'listed': 0.0,
 'participation': 0.0,
 'prerequisites': 0.0,
 'python': 1.0,
 'registration': 0.0,
 'required': 0.0,
 'setup': 0.0,
 'start': 0.0,
 'starts': 0.0,
 'submit': 0.0}

In [16]:
doc_dict = dict(zip(names, X.toarray()[2]))
doc_dict

{'15th': 0.0,
 '2024': 0.0,
 'cloud': 0.0,
 'course': 0.0,
 'date': 0.5,
 'github': 0.0,
 'google': 0.0,
 'homeworks': 0.5,
 'jan': 0.0,
 'listed': 0.0,
 'participation': 0.0,
 'prerequisites': 0.0,
 'python': 0.0,
 'registration': 0.0,
 'required': 0.0,
 'setup': 0.0,
 'start': 0.5,
 'starts': 0.0,
 'submit': 0.5}

In [17]:
df_qd = pd.DataFrame([query_dict, doc_dict], index=['query', 'doc']).T

(df_qd['query'] * df_qd['doc']).sum()

0.0

In [18]:
X.dot(q.T).todense()

matrix([[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.46369322]])

In [27]:
score = cosine_similarity(X, q).flatten()

In [28]:
score

array([0.        , 0.        , 0.        , 0.        , 0.46369322])

In [29]:
np.argsort(score)

array([0, 1, 2, 3, 4])

In [19]:
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}


for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=5)
    X = cv.fit_transform(df[field])

    transformers[field] = cv
    matrices[field] = X

transformers['text'].get_feature_names_out()
matrices['text']

<948x1333 sparse matrix of type '<class 'numpy.float64'>'
	with 23808 stored elements in Compressed Sparse Row format>

In [56]:
n=len(df)
score =np.zeros(n)
query = "I have just discovered the course, is it too late to join?"
boosts={
    'question':3,
    'text':0.5
}
for f in fields:
    q = transformers[f].transform([query])
    X = matrices[f]

    f_score = cosine_similarity(X, q).flatten()

    boost = boosts.get(f, 1.0)
    
    score = score + boost * f_score

In [24]:
query = "I just singned up. Is it too late to join the course?"

q = transformers['text'].transform([query])
score = cosine_similarity(matrices['text'], q).flatten()

score

array([0.3336047 , 0.        , 0.        , 0.1328874 , 0.        ,
       0.        , 0.        , 0.12722114, 0.        , 0.        ,
       0.        , 0.10830554, 0.        , 0.        , 0.        ,
       0.23530268, 0.        , 0.        , 0.04595339, 0.        ,
       0.        , 0.        , 0.22668   , 0.07952931, 0.        ,
       0.        , 0.        , 0.1894954 , 0.08310739, 0.        ,
       0.        , 0.        , 0.        , 0.03724346, 0.        ,
       0.        , 0.        , 0.        , 0.16484429, 0.0231432 ,
       0.03424155, 0.05174621, 0.        , 0.03167699, 0.        ,
       0.        , 0.        , 0.        , 0.02097473, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04722243, 0.        , 0.0073737 ,
       0.        , 0.        , 0.        , 0.        , 0.04161211,
       0.        , 0.        , 0.        , 0.        , 0.     

In [57]:
filters = {
    'course': 'data-engineering-zoomcamp'
}

In [58]:
for field, value in filters.items():
    mask = (df[field]==value).astype(int).values
    score = score * mask

score

array([3.28960182, 3.49512426, 2.70735166, 2.86194785, 3.49512426,
       3.49512426, 1.93689291, 3.58291062, 2.67242848, 3.49512426,
       3.10198469, 2.38161407, 0.49512426, 0.49512426, 0.49512426,
       0.54352887, 0.49512426, 2.63772182, 0.53277026, 0.49512426,
       0.49512426, 0.49512426, 0.64505807, 0.54772763, 0.49512426,
       0.49512426, 0.49512426, 0.63236164, 0.55926629, 0.49512426,
       0.49512426, 0.49512426, 0.49512426, 1.76390631, 3.49512426,
       1.72080809, 0.49512426, 0.49512426, 0.49512426, 0.5109058 ,
       0.51969835, 1.9654566 , 0.49512426, 0.51677312, 0.        ,
       0.        , 0.        , 0.        , 0.01402187, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.03369519, 0.        , 0.00490423,
       0.        , 0.        , 0.        , 0.        , 0.02910051,
       0.        , 0.        , 0.        , 0.        , 0.     

In [61]:
idx = np.argsort(-score)[:5]

In [62]:
df.iloc[idx]

,course,section,question,text
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
5,data-engineering-zoomcamp,General course-related questions,Course - how many Zoomcamps in a year?,"There are 3 Zoom Camps in a year, as of 2024. ..."
34,data-engineering-zoomcamp,General course-related questions,How can we contribute to the course?,Star the repo! Share it with friends if you fi...


In [26]:
mask = (df.course == 'data-engineering-zoomcamp').values
score = score * mask
score

array([0.3336047 , 0.        , 0.        , 0.1328874 , 0.        ,
       0.        , 0.        , 0.12722114, 0.        , 0.        ,
       0.        , 0.10830554, 0.        , 0.        , 0.        ,
       0.23530268, 0.        , 0.        , 0.04595339, 0.        ,
       0.        , 0.        , 0.22668   , 0.07952931, 0.        ,
       0.        , 0.        , 0.1894954 , 0.08310739, 0.        ,
       0.        , 0.        , 0.        , 0.03724346, 0.        ,
       0.        , 0.        , 0.        , 0.16484429, 0.0231432 ,
       0.03424155, 0.05174621, 0.        , 0.03167699, 0.        ,
       0.        , 0.        , 0.        , 0.02097473, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.04722243, 0.        , 0.0073737 ,
       0.        , 0.        , 0.        , 0.        , 0.04161211,
       0.        , 0.        , 0.        , 0.        , 0.     

In [31]:
idx = np.argsort(-score)[:10]
idx

array([  0,  15,  22,  27,  38, 287,   3,   7, 113,  11])

In [32]:
df.iloc[idx].text

0      The purpose of this document is to capture fre...
15     No, late submissions are not allowed. But if t...
22     It's up to you which platform and environment ...
27     You can do most of the course without a cloud....
38     You will have two attempts for a project. If t...
287    This error could result if you are using some ...
3      You don't need it. You're accepted. You can al...
7      Yes, we will keep all the materials after the ...
113    In the join queries, if we mention the column ...
11     No, you can only get a certificate if you fini...
Name: text, dtype: object

In [1]:
class TextSearch:

    def __init__(self, text_fields):
        self.text_fields = text_fields
        self.matrices = {}
        self.vectorizers = {}

    def fit(self, records, vectorizer_params={}):
        self.df = pd.DataFrame(records)

        for f in self.text_fields:
            cv = TfidfVectorizer(**vectorizer_params)
            X = cv.fit_transform(self.df[f])
            self.matrices[f] = X
            self.vectorizers[f] = cv

    def search(self, query, n_results=10, boost={}, filters={}):
        score = np.zeros(len(self.df))

        for f in self.text_fields:
            b = boost.get(f, 1.0)
            q = self.vectorizers[f].transform([query])
            s = cosine_similarity(self.matrices[f], q).flatten()
            score = score + b * s

        for field, value in filters.items():
            mask = (self.df[field] == value).values
            score = score * mask

        idx = np.argsort(-score)[:n_results]
        results = self.df.iloc[idx]
        return results.to_dict(orient='records')

In [2]:
index = TextSearch(
    text_fields=['section', 'question', 'text']
)
index.fit(documents)

index.search(
    query='I just singned up. Is it too late to join the course?',
    n_results=5,
    boost={'question': 3.0},
    filters={'course': 'data-engineering-zoomcamp'}
)

NameError: name 'documents' is not defined

In [25]:
from sklearn.decomposition import TruncatedSVD

X = matrices['text']
cv = transformers['text']

svd = TruncatedSVD(n_components=16)
X_emb = svd.fit_transform(X)

X_emb[0]

array([ 0.09653177, -0.08220603, -0.10124077,  0.07923138,  0.06793655,
       -0.0598059 ,  0.02502459, -0.15197457,  0.25750543,  0.27351015,
       -0.07459887,  0.05803159,  0.10584214, -0.07052828,  0.02173392,
       -0.01862113])

In [26]:
query = 'I just singned up. Is it too late to join the course?'

Q = cv.transform([query])
Q_emb = svd.transform(Q)
Q_emb[0]

array([ 0.05790734, -0.03854519, -0.05547797,  0.02757716,  0.04050848,
       -0.06078484,  0.01604332, -0.09957351,  0.18333541,  0.17069487,
       -0.05509161,  0.0635861 ,  0.08047762, -0.03195769,  0.02845073,
       -0.00894368])

In [27]:
np.dot(X_emb[0], Q_emb[0])

0.15173472270626515

In [29]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
df.loc[idx]

,course,section,question,text
764,machine-learning-zoomcamp,Projects (Midterm and Capstone),What If I submitted only two projects and fail...,If you have submitted two projects (and peer-r...
449,machine-learning-zoomcamp,General course-related questions,The course has already started. Can I still jo...,"Yes, you can. You won’t be able to submit some..."
436,machine-learning-zoomcamp,General course-related questions,Is it going to be live? When?,"The course videos are pre-recorded, you can st..."
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
451,machine-learning-zoomcamp,General course-related questions,Can I submit the homework after the due date?,"No, it’s not possible. The form is closed afte..."
11,data-engineering-zoomcamp,General course-related questions,Certificate - Can I follow the course in a sel...,"No, you can only get a certificate if you fini..."
15,data-engineering-zoomcamp,General course-related questions,Homework - Are late submissions of homework al...,"No, late submissions are not allowed. But if t..."
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
814,mlops-zoomcamp,+-General course questions,What if my answer is not exactly the same as t...,Please choose the closest one to your answer. ...


In [32]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=16)
X_emb = nmf.fit_transform(X)
X_emb[0]

array([0.12969959, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00062932, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [33]:
Q = cv.transform([query])
Q_emb = nmf.transform(Q)
Q_emb[0]

array([0.08619862, 0.00240708, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00244353,
       0.        ])

In [34]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
df.loc[idx]

,course,section,question,text
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
814,mlops-zoomcamp,+-General course questions,What if my answer is not exactly the same as t...,Please choose the closest one to your answer. ...
449,machine-learning-zoomcamp,General course-related questions,The course has already started. Can I still jo...,"Yes, you can. You won’t be able to submit some..."
11,data-engineering-zoomcamp,General course-related questions,Certificate - Can I follow the course in a sel...,"No, you can only get a certificate if you fini..."
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
764,machine-learning-zoomcamp,Projects (Midterm and Capstone),What If I submitted only two projects and fail...,If you have submitted two projects (and peer-r...
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
451,machine-learning-zoomcamp,General course-related questions,Can I submit the homework after the due date?,"No, it’s not possible. The form is closed afte..."
436,machine-learning-zoomcamp,General course-related questions,Is it going to be live? When?,"The course videos are pre-recorded, you can st..."
450,machine-learning-zoomcamp,General course-related questions,When does the next iteration start?,The course is available in the self-paced mode...
